<a href="https://colab.research.google.com/github/nitthapr/ntner/blob/master/03_1_BERT_Thai_NER_v7_6_3_expB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# From : ner-bert/exps/conll2003 BERTBiLSTMCRF.ipynb

# 1) Setting

## 1.1) Load lib and set path

In [1]:
PROJECT_DATA_PATH = '/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER'

pythai_dataset_path = f'{PROJECT_DATA_PATH}/in/train.data'
ner_dataset_path = f'{PROJECT_DATA_PATH}/in/ner_dataset_pythai.csv'
output_model_path = f'{PROJECT_DATA_PATH}/model/'
checkpoint_path = f'{PROJECT_DATA_PATH}/checkpoint/'
bert_thai_model_path = f'{PROJECT_DATA_PATH}/model/bert_base_th/bert_base_th'
bert_multi_model_path = f'{PROJECT_DATA_PATH}/model/multi_cased_L-12_H-768_A-12/bert_multi_cased'

In [2]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, f'{PROJECT_DATA_PATH}/package/sberbank')
#sys.path

# sys.path.remove(sys.path[2])

In [3]:
# Install require package
src_ = f'{PROJECT_DATA_PATH}/package/sberbank/requirements.txt'
print(src_)
!pip3 install -r "/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/package/sberbank/requirements.txt" >/dev/null
!pip3 install git+https://github.com/huggingface/transformers.git --upgrade > /dev/null
!pip3 install pytorch-pretrained-bert >/dev/null
!pip3 install sentencepiece
!pip3 install urllib3==1.25.10

/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/package/sberbank/requirements.txt
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-qjw2yqre
ERROR: botocore 1.20.44 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 1.2MB 15.7MB/s 
     |████████████████████████████████| 133kB 13.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
%load_ext autoreload
%autoreload 2

import os
import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

# 2) Load data for BERT and Train





## 2.1) * Set Data and Output*

In [5]:
p_sample_addpath = '_v2_80-20_main_r1'
p_outsber_addpath = '_80-20_v2_r4' # for add to output/sberbak folder

In [6]:
# copy train test Data to local path
!mkdir "./data"
!gsutil cp -r "/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/wk/pythainer_for_sberbank{p_sample_addpath}/." "./data/."

Copying file:///content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/wk/pythainer_for_sberbank_v2_80-20_main_r1/./test.txt...
Copying file:///content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/wk/pythainer_for_sberbank_v2_80-20_main_r1/./eng.testb...
Copying file:///content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/wk/pythainer_for_sberbank_v2_80-20_main_r1/./eng.train.train.csv...
Copying file:///content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/wk/pythainer_for_sberbank_v2_80-20_main_r1/./train.txt...
/
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/wk/pythainer_for_sberbank_v2_80-20_main_r1/./eng.testa.dev.csv...
Copying file:///con

In [ ]:
"""
##load normal data & model (without char embedding)
from modules.data import bert_thai_data as bert_data2
idx2labels_save_path = "./data/idx2labels.txt"
idx2cls_save_path = "./data/idx2cls.txt"
data = bert_data2.LearnData.create(
    train_df_path="./data/train.txt",
    valid_df_path="./data/test.txt",
    idx2labels_path=idx2labels_save_path,
    idx2cls_path=idx2cls_save_path,
    clear_cache=True,
    #markup='BIO',  #uncomment for BIO training
    batch_size=128
)

#changethis class model if you want to test other experiment
from modules.models.bert_models import BERTBiLSTMCRF
model = BERTBiLSTMCRF.create(label_size=len(data.train_ds.idx2label),model_name='bert-th',lstm_dropout=0.5, crf_dropout=0.5)
model.get_n_trainable_params()
"""

'\n##load normal data & model (without char embedding)\nfrom modules.data import bert_thai_data as bert_data2\nidx2labels_save_path = "./data/idx2labels.txt"\nidx2cls_save_path = "./data/idx2cls.txt"\ndata = bert_data2.LearnData.create(\n    train_df_path="./data/train.txt",\n    valid_df_path="./data/test.txt",\n    idx2labels_path=idx2labels_save_path,\n    idx2cls_path=idx2cls_save_path,\n    clear_cache=True,\n    #markup=\'BIO\',  #uncomment for BIO training\n    batch_size=128\n)\n\n#changethis class model if you want to test other experiment\nfrom modules.models.bert_models import BERTBiLSTMCRF\nmodel = BERTBiLSTMCRF.create(label_size=len(data.train_ds.idx2label),model_name=\'bert-th\',lstm_dropout=0.5, crf_dropout=0.5)\nmodel.get_n_trainable_params()\n'

In [7]:
### use this block instead if want to use BERTModel that have char embedding
### ex: BERTCharBiLSTMCRF
##----Olunlah----
##------- Char2idx Model ------------
import json
CHARIDX_PATH = '/content/drive/My Drive/Colab Notebooks/IS_NER/data/model/Supanut/charidx/char2idx.txt'
#CHARIDX_PATH = '/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/in/LST20/char2idx.txt'
with open(CHARIDX_PATH) as json_file:
    char2idx = json.load(json_file)
##----EndOlunlah------
##load data & model with char embedding
from modules.data import bert_thai_wc_data as bert_data
idx2labels_save_path = "./data/idx2labels.txt"
idx2cls_save_path = "./data/idx2cls.txt"
data = bert_data.LearnData.create(
    train_df_path="./data/train.txt.train.csv",
    valid_df_path="./data/test.txt.dev.csv",
    idx2labels_path=idx2labels_save_path,
    idx2cls_path=idx2cls_save_path,
    clear_cache=True,
    markup='BIO',
    batch_size=16 # original P' lek 64
)
from modules.models.bert_models import BERTCharBiLSTMCRF
#model = BERTCharBiLSTMCRF.create(label_size=len(data.train_ds.idx2label),model_name='bert-th',num_char_dict=len(char2idx),char_lstm_embedding_size=64,char_lstm_hidden_dim=64,lstm_dropout=0.5, char_lstm_dropout=0.5, crf_dropout=0.5)
model = BERTCharBiLSTMCRF.create(label_size=len(data.train_ds.idx2label),model_name='bert-th',num_char_dict=len(char2idx),lstm_dropout=0.5, char_lstm_dropout=0.5, crf_dropout=0.5)
model.get_n_trainable_params()
## check character index
#len(data.train_dl.dataset)
#dt = data.train_dl.dataset[0].data
#data.train_dl.dataset[0].bert_tokens
#dt[4]
#cl = data.train_dl.collate_fn(data.train_dl.dataset[0])

2402072

In [ ]:
#save idx2labels and idx2cls for later use when predict with model train by this data
import os
model_name2save = "BERTCharBiLSTMCRF-IO-7.6.3-expB3"
GGD_save_model_label_path = f"/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/output/sberbank{p_outsber_addpath}/"+model_name2save+"-labels.txt"
GGD_save_model_cls_path = f"/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/output/sberbank{p_outsber_addpath}/"+model_name2save+"-cls.txt"
os.environ['idx2labels_save_path'] = idx2labels_save_path
os.environ['idx2cls_save_path'] = idx2cls_save_path
os.environ['GGD_save_model_label_path'] = GGD_save_model_label_path
os.environ['GGD_save_model_cls_path'] = GGD_save_model_cls_path
!gsutil cp -r "$idx2labels_save_path" "$GGD_save_model_label_path"
!gsutil cp -r "$idx2cls_save_path" "$GGD_save_model_cls_path"

Copying file://./data/idx2labels.txt...
-
Operation completed over 1 objects/232.0 B.                                      
CommandException: No URLs matched: ./data/idx2cls.txt


In [ ]:
#setting model save path & train param
from modules.train.train import NerLearner
num_epochs = 25
#!mkdir "./models"
#save_model_path = "./models/model.cpt"
save_model_path = f"/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/output/sberbank{p_outsber_addpath}/"+model_name2save+".cpt"

In [ ]:
#create learner for model training
learner = NerLearner(
    model, data, save_model_path, lr = 0.0001 , t_total=num_epochs * len(data.train_dl))

# 3) Train

In [ ]:
#if model exist in google drive load model to resume training
import os
if (os.path.exists(save_model_path)):
  learner.load_model(path=save_model_path)

In [ ]:
#train model based on model & learner configuration
learner.fit(epochs=num_epochs)

INFO:root:Resuming train... Current epoch 0.


INFO:root:
epoch 1, average train epoch loss=0.10743



INFO:root:on epoch 0 by max_f1: 0.922
INFO:root:Saving new best model...


                precision    recall  f1-score   support

           B_O      0.980     0.985     0.982     40575
    B_LOCATION      0.861     0.885     0.873       965
    I_LOCATION      0.883     0.848     0.865      1114
B_ORGANIZATION      0.908     0.900     0.904      1146
I_ORGANIZATION      0.904     0.817     0.858      1330
         B_LAW      0.857     0.769     0.811        39
         I_LAW      0.877     0.826     0.851       121
      B_PERSON      0.961     0.923     0.942       613
      I_PERSON      0.979     0.966     0.973      2321
        B_TIME      0.883     0.827     0.854       191
        I_TIME      0.917     0.872     0.894       391
         B_LEN      0.818     0.947     0.878        19
         I_LEN      0.870     0.930     0.899        43
       B_PHONE      0.905     0.950     0.927        20
        B_DATE      0.952     0.930     0.941       401
        I_DATE      0.982     0.954     0.968       846
       B_MONEY      0.979     0.949     0.964  

INFO:root:
epoch 2, average train epoch loss=0.089782



...

INFO:root:on epoch 2 by max_f1: 0.923


                precision    recall  f1-score   support

           B_O      0.980     0.985     0.983     40575
    B_LOCATION      0.876     0.875     0.875       965
    I_LOCATION      0.890     0.846     0.867      1114
B_ORGANIZATION      0.910     0.897     0.903      1146
I_ORGANIZATION      0.897     0.822     0.858      1330
         B_LAW      0.853     0.744     0.795        39
         I_LAW      0.872     0.843     0.857       121
      B_PERSON      0.963     0.925     0.943       613
      I_PERSON      0.978     0.968     0.973      2321
        B_TIME      0.863     0.822     0.842       191
        I_TIME      0.903     0.885     0.894       391
         B_LEN      0.818     0.947     0.878        19
         I_LEN      0.792     0.884     0.835        43
       B_PHONE      0.944     0.850     0.895        20
        B_DATE      0.949     0.933     0.941       401
        I_DATE      0.982     0.959     0.970       846
       B_MONEY      0.959     0.949     0.954  

INFO:root:
epoch 25, average train epoch loss=0.043973



INFO:root:on epoch 2 by max_f1: 0.923


                precision    recall  f1-score   support

           B_O      0.980     0.986     0.983     40575
    B_LOCATION      0.881     0.868     0.875       965
    I_LOCATION      0.904     0.829     0.865      1114
B_ORGANIZATION      0.905     0.902     0.903      1146
I_ORGANIZATION      0.890     0.825     0.856      1330
         B_LAW      0.853     0.744     0.795        39
         I_LAW      0.909     0.826     0.866       121
      B_PERSON      0.963     0.927     0.944       613
      I_PERSON      0.979     0.969     0.974      2321
        B_TIME      0.872     0.822     0.846       191
        I_TIME      0.903     0.885     0.894       391
         B_LEN      0.818     0.947     0.878        19
         I_LEN      0.809     0.884     0.844        43
       B_PHONE      0.944     0.850     0.895        20
        B_DATE      0.961     0.928     0.944       401
        I_DATE      0.984     0.957     0.971       846
       B_MONEY      0.959     0.949     0.954  

In [ ]:
#save train result to file
!mkdir "./models"
import pandas as pd
pd.set_option('display.max_colwidth',3000)
result_data = {'train_losses': learner.train_losses, 'test_losses': learner.test_losses, 'history': learner.history}
result_data_df = pd.DataFrame(data=result_data)
save_model_result_path = "./models/model-result.csv"
result_data_df.to_csv(save_model_result_path, index=False)

#copy result to google drive
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
import os
GGD_save_model_result_path = "/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/output/sberbank{p_outsber_addpath}"+model_name2save+"-result-"+timestr+".csv"
os.environ['save_model_result_path'] = save_model_result_path
os.environ['GGD_save_model_result_path'] = GGD_save_model_result_path
!gsutil cp -r "$save_model_result_path" "$GGD_save_model_result_path"


mkdir: cannot create directory ‘./models’: File exists
Copying file://./models/model-result.csv...
/ [1 files][ 84.1 KiB/ 84.1 KiB]                                                
Operation completed over 1 objects/84.1 KiB.                                     


In [ ]:
result_data_df

,train_losses,test_losses,history
0,163.764293,96.888786,precision recall f1-score support\n\n B_O 0.645 0.991 0.781 10714\n B_DATE 0.000 0.000 0.000 100\n I_DATE 0.000 0.000 0.000 203\n B_TIME 0.000 0.000 0.000 41\n I_TIME 0.000 0.000 0.000 77\n B_PERSON 0.000 0.000 0.000 160\n I_PERSON 0.000 0.000 0.000 576\nB_ORGANIZATION 0.000 0.000 0.000 301\n B_URL 0.000 0.000 0.000 5\n I_URL 0.000 0.000 0.000 56\nI_ORGANIZATION 0.000 0.000 0.000 423\n B_PERCENT 0.000 0.000 0.000 7\n I_PERCENT 0.000 0.000 0.000 12\n B_LOCATION 0.005 0.005 0.005 222\n I_LOCATION 0.000 0.000 0.000 291\n B_MONEY 0.000 0.000 0.000 26\n I_MONEY 0.000 0.000 0.000 67\n B_PHONE 0.000 0.000 0.000 6\n B_LAW 0.000 0.000 0.000 11\n I_LAW 0.000 0.000 0.000 48\n I_PHONE 0.000 0.000 0.000 16\n B_LEN 0.000 0.000 0.000 4\n I_LEN 0.000 0.000 0.000 10\n B_ZIP 0.000 0.000 0.000 2\n B_EMAIL 0.000 0.000 0.000 1\n I_EMAIL 0.000 0.000 0.000 6\n\n micro avg 0.636 0.793 0.706 13385\n macro avg 0.025 0.038 0.030 13385\n weighted avg 0.516 0.793 0.625 13385\n
1,69.731238,43.608783,precision recall f1-score support\n\n B_O 0.806 0.958 0.875 10714\n B_DATE 0.667 0.080 0.143 100\n I_DATE 0.556 0.365 0.440 203\n B_TIME 0.000 0.000 0.000 41\n I_TIME 0.000 0.000 0.000 77\n B_PERSON 0.700 0.044 0.082 160\n I_PERSON 0.615 0.590 0.602 576\nB_ORGANIZATION 0.394 0.086 0.142 301\n B_URL 0.000 0.000 0.000 5\n I_URL 0.688 0.196 0.306 56\nI_ORGANIZATION 0.050 0.002 0.005 423\n B_PERCENT 0.000 0.000 0.000 7\n I_PERCENT 0.000 0.000 0.000 12\n B_LOCATION 0.071 0.005 0.008 222\n I_LOCATION 0.379 0.038 0.069 291\n B_MONEY 0.000 0.000 0.000 26\n I_MONEY 0.000 0.000 0.000 67\n B_PHONE 0.000 0.000 0.000 6\n B_LAW 0.000 0.000 0.000 11\n I_LAW 0.000 0.000 0.000 48\n I_PHONE 0.000 0.000 0.000 16\n B_LEN 0.000 0.000 0.000 4\n I_LEN 0.000 0.000 0.000 10\n B_ZIP 0.000 0.000 0.000 2\n B_EMAIL 0.000 0.000 0.000 1\n I_EMAIL 0.000 0.000 0.000 6\n\n micro avg 0.790 0.803 0.797 13385\n macro avg 0.189 0.091 0.103 13385\n weighted avg 0.716 0.803 0.742 13385\n
2,31.751785,18.624202,precision recall f1-score support\n\n B_O 0.914 0.964 0.938 10714\n B_DATE 0.739 0.650 0.691 100\n I_DATE 0.797 0.872 0.833 203\n B_TIME 0.714 0.244 0.364 41\n I_TIME 0.788 0.532 0.636 77\n B_PERSON 0.917 0.694 0.790 160\n I_PERSON 0.898 0.839 0.867 576\nB_ORGANIZATION 0.700 0.668 0.684 301\n B_URL 0.000 0.000 0.000 5\n I_URL 0.750 0.964 0.844 56\nI_ORGANIZATION 0.682 0.345 0.458 423\n B_PERCENT 0.000 0.000 0.000 7\n I_PERCENT 1.000 0.083 0.154 12\n B_LOCATION 0.477 0.523 0.499 222\n I_LOCATION 0.552 0.381 0.451 291\n B_MONEY 0.581 0.692 0.632 26\n I_MONEY 0.855 0.881 0.868 67\n B_PHONE 0.000 0.000 0.000 6\n B_LAW 0.000 0.000 0.000 11\n I_LAW 0.000 0.000 0.000 48\n I_PHONE 0.000 0.000 0.000 16\n B_LEN 0.000 0.000 0.000 4\n I_LEN 0.000 0.000 0.000 10\n B_ZIP 0.000 0.000 0.000 2\n B_EMAIL 0.000 0.000 0.000 1\n I_EMAIL 0.000 0.000 0.000 6\n\n micro avg 0.886 0.891 0.888 13385\n macro avg 0.437 0.359 0.373 13385\n weighted avg 0.872 0.891 0.878 13385\n
3,17.210355,12.336667,precision recall f1-score support\n\n B_O 0.953 0.961 0.957 10714\n B_DATE 0.777 0.870 0.821 100\n I_DATE 0.830 0.985 0.901 203\n B_TIME 0.721 0.756 0.738 41\n I_TIME 0.791 0.883 0.834 77\n B_PERSON 0.955 0.800 0.871 160\n I_PERSON 0.966 0.898 0.931 576\nB_ORGANIZATION 0.833 0.761 0.795 301\n B_URL 1.000 0.800 0.889 5\n I_URL 0.862 1.000 0.926 56\nI_ORGANIZATION 0.825 0.615 0.705 423\n B_PERCENT 1.000 0.429 0.600 7\n I_PERCENT 0.333 0.083 0.133 12\n B_LOCATION 0.726 0.752 0.739 222\n I_LOCATION 0.766 0.540 0.633 291\n B_MONEY 0.490 0.923 0.640 26\n I_MONEY 0.759 0.940 0.840 67\n B_PHONE 1.000 0.500 0.667 6\n B_LAW 0.000 0.000 0.000 11\n I_LAW 1.000 0.062 0.118 48\n I_PHONE 0.833 0.938 0.882 16\n B_LEN 1.000 0.250 0.400 4\n I_LEN 0.500 0.400 0.444 10\n B_ZIP 0.000 0.000 0.000 2\n B_EMAIL 0.000 0.000 0.000 1\n I_EMAIL 0.000 0.000 0.000 6\n\n micro avg 0.932 0.920 0.926 13385\n macro avg 0.689 0.583 0.595 13385\n weighted avg 0.930 0.920 0.922 13385\n
4,11.419505,8.926666,prec

In [ ]:
#copy model, model result & model's label index to  Google Drive
#model_name2save = "BERTCharBiLSTMCRF-IO-7.6.2-LST20-3"
#GGD_save_model_path = "/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/output/sberbank/"+model_name2save+".cpt"
#os.environ['save_model_path'] = save_model_path
#os.environ['GGD_save_model_path'] = GGD_save_model_path
#!gsutil cp -r "$save_model_path" "$GGD_save_model_path"

Copying file://./models/model-result.csv...
/ [1 files][ 10.8 KiB/ 10.8 KiB]                                                
Operation completed over 1 objects/10.8 KiB.                                     


In [ ]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [ ]:
def _to_one_hot(y, num_classes):
    scatter_dim = len(y.size())
    y_tensor = y.view(*y.size(), -1)
    zeros = torch.zeros(*y.size(), num_classes, dtype=y.dtype)
        
    return zeros.scatter(scatter_dim, y_tensor, 1)

In [ ]:
dt = data.train_dl.dataset[0].data

data.train_dl.dataset[0].bert_tokens
import torch
import numpy as np
char_vec = torch.LongTensor(dt[4])

arr = []
res = torch.LongTensor(char_vec.shape[0], char_vec.shape[1] , len(char2idx))
for char_seq in char_vec:
  x = _to_one_hot(char_seq, len(char2idx))
  arr.append(x.unsqueeze(0))
result = torch.cat(arr, out=res)
embedding_size = 32
hidden_dim = 32
charembeddings = torch.nn.Embedding(len(char2idx), embedding_size)
lstm = torch.nn.LSTM(embedding_size, hidden_dim, bidirectional=True, batch_first=True, dropout=0.5)
embedered_char = charembeddings(result)
embedered_char.shape

char_vec2 = torch.empty(embedered_char.shape[0], embedered_char.shape[1], embedered_char.shape[3]*2).cuda()
for idx, ch in enumerate(embedered_char):
    s_ch_rep, _ = lstm(ch)
    s_ch_rep_f = s_ch_rep[:, -1, 0: embedding_size]
    s_ch_rep_b = s_ch_rep[:, 0, embedding_size:]
    s_ch_rep = torch.cat((s_ch_rep_f, s_ch_rep_b), dim=1)
    char_vec2[idx] = s_ch_rep
print(char_vec2.shape)

torch.Size([424, 30, 189])


RuntimeError: ignored

#Predict & using

In [ ]:
#set path and install required lib
import sys
# insert at 1, 0 is the script path (or '' in REPL)
PROJECT_DATA_PATH = '/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER'
sys.path.insert(1, f'{PROJECT_DATA_PATH}/package/sberbank')
sys.path
!pip3 install git+https://github.com/huggingface/transformers.git --upgrade > /dev/null
!pip3 install pytorch-pretrained-bert >/dev/null
!pip3 install urllib3==1.25.10
!pip install pythainlp

In [ ]:
#copy model and model's label file to local
model_name2use = "BERTBiLSTMAttnCRF-IO-7.4.3-95-05-set3"
model_file_path = f"/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/output/sberbank{p_outsber_addpath}/"+model_name2use+".cpt"
model_label_path = f"/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/output/sberbank{p_outsber_addpath}/"+model_name2use+"-labels.txt"
model_load2local_path = "./models/model.cpt"
idx2labels_load2local_path = './data/idx2labels.txt'
import os
os.environ['model_file_path'] = model_file_path
os.environ['model_label_path'] = model_label_path
os.environ['model_load2local_path'] = model_load2local_path
os.environ['idx2labels_load2local_path'] = idx2labels_load2local_path
!gsutil cp -r "$model_file_path" "$model_load2local_path"
!gsutil cp -r "$model_label_path" "$idx2labels_load2local_path"

Copying file:///content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/output/sberbank/BERTBiLSTMAttnCRF-IO-7.4.3-95-05-set3.cpt...
-
Operation completed over 1 objects/411.6 MiB.                                    
Copying file:///content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/output/sberbank/BERTBiLSTMAttnCRF-IO-7.4.3-95-05-set3-labels.txt...
/ [1 files][  131.0 B/  131.0 B]                                                
Operation completed over 1 objects/131.0 B.                                      


In [ ]:
#copy predict data or create the data frame
#create data frame 
import pandas as pd 
pd.set_option('display.max_colwidth',1000)
from pythainlp.tokenize import word_tokenize
text = 'มีการลงนามแล้ว “สัญญา 2.3” โครงการความร่วมมือระหว่าง รัฐบาลไทย และ รัฐบาลจีน ในการพัฒนาระบบรถไฟความเร็วสูง เพื่อเชื่อมโยงกรุงเทพมหานคร - นครราชสีมา มูลค่ากว่า 5 หมื่นล้านบาท เมื่อวันพุธ (28 ตุลาคม 2563) ที่ผ่านมา โดยเป็นการลงนามระหว่าง 3 ฝ่าย ได้แก่ การรถไฟแห่งประเทศไทย (ร.ฟ.ท.) , บริษัท ไชน่า เรลเวย์ อินเตอร์เนชันแนล และ บริษัทไชน่า เรลเวย์ ดีไซน์ คอร์ปอเรชัน'
tktext = word_tokenize(text)
empty_label = []
for t in tktext:
  empty_label.append('O')
d = {'labels': [" ".join(empty_label)], 'text': [" ".join(tktext)], 'cls': [False]}
df = pd.DataFrame(data=d)
df_path = ""

#df.loc[0,:]
#df

In [ ]:
"""
##or copy data to data frame path
df_path = "./data/test.txt.dev.csv"
!gsutil cp -r "/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/wk/pythainer_for_sberbank/test.txt.dev.csv" df_path
df = None
"""

In [ ]:
#load & count model's labels index
import json
label_count = 0
with open(idx2labels_load2local_path, "r", encoding="utf-8") as f:
  for line in f:
    if (line != ''):
      label_count += 1
label_count = label_count

In [ ]:
#load model
from modules.models.bert_models import BERTBiLSTMAttnCRF
model4predict = BERTBiLSTMAttnCRF.create(label_size=label_count,model_name='bert-th',lstm_dropout=0.5, crf_dropout=0.5)

import torch
model4predict.load_state_dict(torch.load(model_load2local_path))
#model4predict.eval()

In [ ]:
#create TextDataSet for  predict data
from modules.data import tokenization as ThaiTokenization
bpe_vocab_path = f'/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/model/th_wiki_bpe/th.wiki.bpe.op25000.vocab'
bpe_model_path = f'/content/drive/My Drive/Colab Notebooks/IS_NER/data/03_BERT_Thai_NER/model/th_wiki_bpe/th.wiki.bpe.op25000.model'
thTokenizer = ThaiTokenization.ThaiTokenizer(vocab_file=bpe_vocab_path, spm_file=bpe_model_path)

from modules.data.bert_thai_data import TextDataSet, TextDataLoader
ds = TextDataSet.create(
        idx2labels_path=idx2labels_load2local_path,
        df=df,df_path=df_path, tokenizer=thTokenizer, clear_cache=False)
dl = TextDataLoader(ds, device='cuda', batch_size=16, shuffle=False)
#ds[0].bert_tokens
#ds[0].tokens
#ds[0].tok_map

In [ ]:
#feed forward to predict y
all_results = []
ds_idx=0
for batch_idx, batch in enumerate(dl):
  output = model4predict.forward(batch)
  for o in enumerate(output[0]):
    o_idx, o_out = o
    out_label = []
    for y in o_out:
      out_label.append(ds.idx2label[y.item()])
    ix = 1
    current_results = []
    for tkp in ds[ds_idx].tok_map:
      if (tkp != -1):
        current_result = ds[ds_idx].tokens[ix], out_label[tkp+1]
        current_results.append(current_result)
      ix += 1
    all_results.append(current_results)
    ds_idx += 1

In [ ]:
all_results

[[('ขนส่ง', 'I_O'),
  ('เปิด', 'I_O'),
  ('ช่องทาง', 'I_O'),
  ('รับฟัง', 'I_O'),
  ('ความเห็น', 'I_O'),
  ('ประชาชน', 'I_O'),
  ('เรื่อง', 'I_O'),
  ('กม.', 'I_O'),
  ('ใบขับขี่', 'I_O'),
  ('_', 'I_O'),
  ('ระหว่าง', 'I_O'),
  ('28', 'I_DATE'),
  ('_', 'I_DATE'),
  ('ส.ค.', 'I_DATE'),
  ('–', 'I_O'),
  ('_', 'I_O'),
  ('11', 'I_DATE'),
  ('ก.ย.', 'I_DATE'),
  ('61', 'I_DATE'),
  ('ย้ำ', 'I_O'),
  ('คน', 'I_O'),
  ('ไม่', 'I_O'),
  ('มี', 'I_O'),
  ('ใบขับขี่', 'I_O'),
  ('ยัง', 'I_O'),
  ('เป็น', 'I_O'),
  ('จำเลย', 'I_O'),
  ('ที่', 'I_O'),
  ('สร้าง', 'I_O'),
  ('สถิติ', 'I_O'),
  ('อุบัติเหตุ', 'I_O'),
  ('สูงสุด', 'I_O')],
 [('เครือข่าย', 'I_ORGANIZATION'),
  ('วิทยุ', 'I_ORGANIZATION'),
  ('ชุมชน', 'I_ORGANIZATION'),
  ('ภาคเหนือ', 'I_ORGANIZATION'),
  ('_', 'I_O'),
  ('17', 'I_ORGANIZATION'),
  ('_', 'I_O'),
  ('จังหวัด', 'I_O'),
  ('เครือข่าย', 'I_O'),
  ('ชุมชนเมือง', 'I_ORGANIZATION'),
  ('เชียงใหม่', 'I_ORGANIZATION'),
  ('ได้', 'I_O'),
  ('ออก', 'I_O'),
  ('แถลงการณ์', 'I_